In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8308947 , -0.03052019, -0.555592  ,  0.64669394],
        [ 0.1740484 ,  0.9626463 ,  0.20741071, -0.1433228 ],
        [ 0.5285084 , -0.26903635,  0.8051697 ,  0.07163957],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [6]:
# add env
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]})
# add robot collision
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

Please create a subscriber to the marker


In [7]:
floor = gtem_dict["floor"]
track = gtem_dict["track"]
floor.set_offset_tf(center= tuple(np.subtract(floor.center, (0,0,0.002))))
track.set_offset_tf(center= tuple(np.subtract(track.center, (0,0,0.005))))
gscene.set_rviz()

In [8]:
wall1 = gscene.create_safe(GEOTYPE.BOX, "wall1", "base_link", (track.dims[0],0.01,0.1), (0,0.1,0.),rpy=(0,0,0), 
                         color=(0.8,0.8,0.8,0.3), display=True, fixed=True, collision=True, parent="track")
wall2 = gscene.create_safe(GEOTYPE.BOX, "wall2", "base_link", (track.dims[0],0.01,0.1), (0,0-0.1,0.),rpy=(0,0,0), 
                         color=(0.8,0.8,0.8,0.3), display=True, fixed=True, collision=True, parent="track")

## add movable

In [9]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [10]:
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.011), (-0.1,0,0.),rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="track")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.011), (0.3,0.0,0), rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="track")

## add brush

In [11]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.05,0.05,0.01), center=(0,0,0.03), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=False, fixed=False, parent="box1")

## init planning scene

In [12]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [13]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [14]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [15]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,0.005), rpy=(np.pi,0,0))

## add objects

In [16]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [17]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
# box2= pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [18]:
box1_top_p = box1.action_points_dict["top_p"]

In [19]:
box1_top_p.set_point_rpy(tuple(np.add(box1_top_p.point, (0,0,0.01))), box1_top_p.rpy_point)
pscene.update_subjects()

In [20]:
sweep = pscene.create_subject(oname="sweep", gname="track", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

### planning pipeline

In [21]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## ui

In [22]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


## Test plan

In [23]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.update_state(crob.home_pose)

   Use a production WSGI server instead.
 * Debug mode: off


In [24]:
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=True, dt_vis=0.01, timeout_loop=1000, N_search=10000, multiprocess=False)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = success
Remaining:4->3 / branching: 0->1 (0.84/1000.0 s, steps/err: 19(425.78792572 ms)/2.79670199886)
node: ('grip0', 0)->('goal', 0) = success
Remaining:3->2 / branching: 1->2 (1.03/1000.0 s, steps/err: 8(194.698095322 ms)/0.00133427273808)
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 1) = fail
node: ('goal', 0)->('goal', 

node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 0)->('goal', 0) = fail
node: ('grip0', 0)->('goal', 0) = fail
node: ('grip0', 0)->('goal', 0) = success
Remaining:3->2 / branching: 1->8 (11.55/1000.0 s, steps/err: 42(933.706998825 ms)/0.00

node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = success
Remaining:2->1 / branching: 9->13 (20.05/1000.0 s, steps/err: 35(766.383886337 ms)/0.00126764250256)
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)

node: ('grip0', 1)->('goal', 1) = success
Remaining:2->1 / branching: 16->18 (29.51/1000.0 s, steps/err: 21(471.957921982 ms)/0.00145540547956)
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->(

node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = fail
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('goal', 1) = success
Remaining:2->1 / branching: 21->24 (38.73/1000.0 s, steps/err: 17(400.923013687 ms)/0.00177926017647)
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
node: ('goal', 1)->('goal', 2) = fail
no

In [25]:
ppline.play_schedule(snode_schedule, period=0.1)

## Text execution

In [26]:
crob.reset_connection([True])

connection_list
[True]
Connect: Server IP (192.168.0.63)


## unsynced execution

In [27]:
time.sleep(5)

In [28]:
ppline.execute_schedule(snode_schedule, vel_scale=0.3)

binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)


KeyboardInterrupt: 

## synced execution

In [ ]:
## 
# with RvizPublisher(gscene, []) as rpub:
#     ppline.execute_schedule(snode_schedule, rviz_pub=rpub)

## Disconnect stereo

In [ ]:
stereo.disconnect()